## 1.Import libraries
The required libraries for this notebook are pandas, sklearn, numpy and matplotlib.

In [107]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import metrics

## 2.1 Edit label.csv data
Add column for binary label: 0 for no tumor, 1 for tumor

In [51]:
df = pd.read_csv('./dataset/label.csv')
df.head()

,file_name,label
0,IMAGE_0000.jpg,meningioma_tumor
1,IMAGE_0001.jpg,no_tumor
2,IMAGE_0002.jpg,meningioma_tumor
3,IMAGE_0003.jpg,glioma_tumor
4,IMAGE_0004.jpg,meningioma_tumor


## 2.2 Add image data
Add image data in the form of an array to the label data and save it in a new pickle file so it does not need to run everytime

In [52]:
def create_binary(df):
    binary = []
    for label in df['label']:
        if 'no_tumor' in label:
            binary.append(0)
        else:
            binary.append(1)
    return binary

In [53]:
def extract_hog(data_path, df):
    hog_features = []
    for filename in df['file_name']:
        # use imread to load image from specified file name
        im = imread(os.path.join(data_path, filename), as_gray = True)
        # use resize the image to a 128 by 64 pixel image
        resized_im = resize(im, (128, 64))
        # Extract Histogram of Oriented Gradients (HOG) for the image
        fd = hog(resized_im, orientations = 9, pixels_per_cell = (8, 8),
                            cells_per_block = (2, 2), visualize = False, block_norm='L2-Hys')
        hog_features.append(fd)
    return hog_features

In [108]:
def PCAPredict(X_train, X_test, k):
    '''
    Inputs
        X: dataset;
        k: number of clusters.
        
    Return
        X_reduced: The array of pixels (i.e. the image) after applying PCA to reduce its dimensions
    '''

    # the built-in function for PCA
    pca = PCA(n_components=k)
    
    # fit the algorithm with dataset
    pca.fit(X_train)
    X_train_PCA = pca.transform(X_train)
    X_test_PCA = pca.transform(X_test)
    
    return X_train_PCA, X_test_PCA

In [127]:
def flatten_image(data_path, df):
    image_data = []
    for filename in df['file_name']:
        # use imread to load image from specified file name
        im = imread(os.path.join(data_path, filename), as_gray = True)
        # use resize the image to a 128 by 64 pixel image
        resized_im = resize(im, (128, 64))
        # Extract Histogram of Oriented Gradients (HOG) for the image
        flattened_image = resized_im.flatten()
        image_data.append(flattened_image)
    return image_data

In [128]:
if os.path.exists('./dataset/label_taskA_hog.pkl'):
    print('The pickle file for Task A with hog features data already exists')
else:
    data_path = './dataset/image'
    binary = create_binary(df)
    hog_features = extract_hog(data_path, df)
    
    label_taskA = pd.DataFrame(data = hog_features)
    label_taskA['binary_label'] = binary
    label_taskA.to_pickle("./dataset/label_taskA_hog.pkl")

The pickle file for Task A with hog features data already exists


In [129]:
if os.path.exists('./dataset/label_taskA_flatten.pkl'):
    print('The pickle file for Task A with flattened image data already exists')
else:
    data_path = './dataset/image'
    binary = create_binary(df)
    image_data = flatten_image(data_path, df)

    label_taskA = pd.DataFrame(data = image_data)
    label_taskA['binary_label'] = binary
    label_taskA.to_pickle("./dataset/label_taskA_flatten.pkl")

In [128]:
# if os.path.exists('./dataset/label_taskA.pkl'):
#     print('The pickle file with image data already exists')
# else:
#     data_path = './dataset/image'
#     # create an empty column to store image data
#     # df['data'] = df['data'].astype(object)
#     hog_features = []
#     for filename in df['file_name']:
#         # use imread to load image from specified file name
#         im = imread(os.path.join(data_path, filename))
#         # use resize the image to a 128 by 64 pixel image
#         resized_im = resize(im, (128, 64))
#         # Extract Histogram of Oriented Gradients (HOG) for the image
#         fd = hog(resized_im, orientations = 9, pixels_per_cell = (8, 8),
#                             cells_per_block = (2, 2), visualize = False, block_norm='L2-Hys')
#         # find index/row number of image file name
#         # idx = df.loc[df['file_name'] == filename].index[0]
#         # save HOG information in 'data' column
#         # df.loc[idx, 'data'] = [fd] 
#         # this resulted in an error: Must have equal len keys and value when setting with an ndarray
#         hog_features.append(fd)
        
#     hog_features
#     df.to_pickle("./dataset/label_taskA.pkl")

The pickle file with image data already exists


In [130]:
feature = 'flatten'
usingPCA = True

In [131]:
if feature == 'hog':
    # Doing classification with image HOG features
    label_taskA = pd.read_pickle("./dataset/label_taskA_hog.pkl")
elif feature == 'flatten':
    # Doing classification with flattened image data with PCA
    label_taskA = pd.read_pickle("./dataset/label_taskA_flatten.pkl")

In [132]:
# # use np.unique to get all unique values in the list of labels
# labels = np.unique(df_edited['label'])

# # set up the matplotlib figure and axes, based on the number of labels
# fig, axes = plt.subplots(1, len(labels))
# fig.set_size_inches(15,4)
# fig.tight_layout()
 
# # make a plot for every label (equipment) type. The index method returns the 
# # index of the first item corresponding to its search string, label in this case
# for ax, label in zip(axes, labels):
#     idx = df_edited.loc[df_edited['label'] == label].index[0]
#     ax.imshow(df_edited.loc[idx, 'data'])
#     ax.axis('off')
#     ax.set_title(label)

In [133]:
X = label_taskA.drop('binary_label',axis=1) # All other features
Y = label_taskA['binary_label'] # binary label

In [141]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 3) 
#test_size= should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split
#everytime you run it without specifying random_state, you will get a different result, this is expected behavior
#print (len(X_test), len(y_test))

print('train set: {}  | test set: {}'.format(round(((len(y_train)*1.0)/len(X)),3),
                                                       round((len(y_test)*1.0)/len(X),3)))
if usingPCA == True:
    k = 10
    X_train, X_test = PCAPredict(X_train, X_test, k)

train set: 0.7  | test set: 0.3


In [142]:
def KNNClassifier(X_train, y_train, X_test,k):

    #Create KNN object with a K coefficient
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, y_train) # Fit KNN model


    Y_pred = neigh.predict(X_test)
    return Y_pred

# clf = svm.SVC()
# clf.fit(X_train, y_train)
# Y_pred = clf.predict(X_test)
# throws error: setting an array element with a sequence
# when getting X from dataframe, the resulting array is dtype object,
# but when getting X directly from hog_features, resulting array is dtype float??

In [143]:
def SVMClassifier(X_train, y_train, X_test):

    #Create SCV object with a K coefficient
    clf = svm.SVC()
    clf.fit(X_train, y_train) # Fit KNN model
    Y_pred = clf.predict(X_test)

    return Y_pred

In [144]:
Y_pred_KNN = KNNClassifier(X_train, y_train, X_test, 4)
score_KNN = metrics.accuracy_score(y_test, Y_pred_KNN)

Y_pred_SVM = SVMClassifier(X_train, y_train, X_test)
score_SVM = metrics.accuracy_score(y_test, Y_pred_SVM)

print('KNN classifier score:', score_KNN)
print('SVM classifier score:', score_SVM)

KNN classifier score: 0.9166666666666666
SVM classifier score: 0.8977777777777778
